# Homework 4

**Name:** -- Diana Zepeda Tatengo --

**e-mail:** -- diana.zepeda6085@alumnos.udg.mx --

# MODULES

In [3]:
# Load modules
import numpy as np
import matplotlib.pyplot as plt
import math

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.stats import exponweib
from scipy.spatial import distance

import plotly.graph_objects as go
import pandas as pd

import panel as pn
pn.extension()

import panel.widgets as pnw
import hvplot.pandas

# CLASSES

In [5]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Multiplication
    def __mul__(self, scalar):
        return Vec2d(self.x * scalar, self.y * scalar)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# FUNCTIONS

In [7]:
#############################
# Brownian motion trajectoy #
#############################
def bm_2d(n_steps=1000, speed=5, s_pos=[0,0]):
    """
    Arguments:
        n_steps: int, number of steps in the simulation (default is 1000).
        speed: float, magnitude of the step in each iteration (default is 5).
        s_pos: list, initial position [x, y] (default is [0,0]).
    Returns:
        BM_2d_df: returns a DataFrame containing the 2D positions at each step
    """
    # Init velocity vector
    velocity = Vec2d(speed,0)
    
    BM_2d_df = np.ones(shape=(n_steps,2))*s_pos
    # Initial position
    BM_2d_df[0] = s_pos
    
    # Generate the trajectory
    for i in range(1,n_steps):        
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)
    
        BM_2d_df[i,0] = BM_2d_df[i-1,0] + velocity.x
        BM_2d_df[i,1] = BM_2d_df[i-1,1] + velocity.y 
        
    return BM_2d_df
####################################
# Correlated Random Walk trajectoy #
####################################
def CRW_2d(n_steps=1000, speed=5, s_pos=[0,0], cauchy=0.5):
    """
    Arguments:
        n_steps: int, number of steps in the simulation (default is 1000).
        speed: float, magnitude of the step in each iteration (default is 5).
        s_pos: list, initial position [x, y] (default is [0,0]).
        cauchy: float, alpha value to get the turn_angle from cauchy (default is 0.5)
    Returns:
        CRW_2d: returns a DataFrame containing the 2D positions at each step
    """
    # Init velocity vector
    velocity = Vec2d(speed,0)       
       
    # Correlation Random Walk in 2d
    CRW_2d = np.ones(shape=(n_steps,2))*s_pos   
    # Initial position
    CRW_2d[0] = s_pos
    
    # Generate the trajectory
    for i in range(1,n_steps):   
        turn_angle = wrapcauchy.rvs(cauchy) # rvs gets random values from the distribution
        velocity = velocity.rotated(turn_angle)
    
        # Displace the particle
        CRW_2d[i,0] = CRW_2d[i-1,0] + velocity.x
        CRW_2d[i,1] = CRW_2d[i-1,1] + velocity.y 
        
    return CRW_2d


###############
# Path Length #
###############
def path_length(trajectory):
    """
    Calculate the total path length of a given trajectory.
    
    Parameters:
    trajectory: A pandas DataFrame containing the trajectory
    
    Returns:
    path_length: A numpy array containing the cumulative sum of the distances
    """
    # Get the Euclidean Distance
    distances = np.linalg.norm(trajectory[1:] - trajectory[:-1], axis=1)
    # Get the Cumulative Sum of the stephs
    return np.cumsum(distances)

#############################
# Mean Squared Displacement #
#############################
def msd(trajectory):
    """
    Compute the Mean Squared Displacement (MSD) for a given trajectory.
    
    Parameters:
    trajectory: A numpy array containing the trajectory
    
    Returns:
    msd: The MSD as a function of time.
    """
    N = len(trajectory)    
    msd = np.zeros(N-1)
    
    for i in range(1,N):
        displacements = trajectory[i:] - trajectory[:N - i]
        squared_displacements = np.sum(displacements**2, axis=1) # Square each coordinate of the displacement and add them together
        msd[i-1] = np.mean(squared_displacements) # Save the msd
    
    return msd

####################################
#  Get the angles between vectors  #
####################################
def trajectory_angles(trajectory):
    """
    Compute the Turning Angles for a given trajectory.
    
    Parameters:
    trajectory: A pandas DataFrame containing the trajectory
    
    Returns:
    angles: A pandas Series containing the angles
    """
    angles = []

    for i in range(1, len(trajectory) - 1):
        # Get Vectors
        V1 = np.array(trajectory[i] - trajectory[i - 1]) # v(i) - v(i-1)
        V2 = np.array(trajectory[i + 1] - trajectory[i]) # v(i+1) - v(i)  
        
        # Dot Product
        dot_product = np.dot(V1, V2)
        
        # Vector magnitudes
        norm_V1 = np.linalg.norm(V1)
        norm_V2 = np.linalg.norm(V2)
        
        # Avoid div by 0
        if norm_V1 == 0 or norm_V2 == 0:
            angles.append(0)
            continue
        
        # Calculate the angle     
        cos_theta = dot_product / (norm_V1 * norm_V2)        
                
        theta = np.arccos(cos_theta)  # Cos -1  get angles on [0, 2π]

        # Convert V1 and V2 to 3D (to apply the cross product)
        V1_3D = np.array([V1[0], V1[1], 0], dtype=np.float64)
        V2_3D = np.array([V2[0], V2[1], 0], dtype=np.float64)
        
        # Calculate the cross product in 3D to determine the sign of the angle
        cross_product = np.cross(V1_3D, V2_3D)[-1]
        
        # If cross product is negative, the angle is negative
        if cross_product < 0:
            theta = -theta            
        
        angles.append(theta)  
    
    return angles

###############
#  Lévy Walk  #
###############
def LevyW_2d(n_steps=1000, speed=5, s_pos=[0,0], alpha=1, beta=1, cauchy=0.7, loc=1):
    """
    Arguments:
        n_steps: int, number of steps in the simulation (default is 1000).
        speed: float, magnitude of the step in each iteration (default is 5).
        s_pos: list, initial position [x, y] (default is [0,0]).
        alpha: float, the stability parameter for the Lévy distribution (default is 1).
        beta: float, the skewness parameter for the Lévy distribution (default is 1).
        cauchy: float, the parameter controlling the dispersion of the Cauchy distribution for the turn angle (default is 0.7).
    Returns:
        Levy_2d: returns a DataFrame containing the 2D positions at each step
    """
    # Init velocity vector
    velocity = Vec2d(speed,0)
    
    # Lévy Flight in 3d
    Levy_flight = np.ones(shape=(n_steps,2))*s_pos
    Levy_flight[0] = s_pos

    # Generate the trajectory
    for i in range(1,n_steps):   
        # Choose a random angle from cauchy distribution
        turn_angle = wrapcauchy.rvs(cauchy)
    
        # Generate step length from lévy distribution
        step_length = levy_stable.rvs(alpha,beta,loc=loc) #loc 1 to get only positive values
    
        velocity = velocity.rotated(turn_angle)
    
        # Calculate the displacement
        displacement = velocity * step_length
    
        # Displace the particle
        Levy_flight[i, 0] = Levy_flight[i-1, 0] + displacement.x
        Levy_flight[i, 1] = Levy_flight[i-1, 1] + displacement.y
        
    return Levy_flight

#################
#  Step Length  #
#################
def step_length(trajectory):
    """
    Calculates the step lengths from a trajectory.

    Parameters:
    trajectory: pandas DataFrame with 'x_pos' and 'y_pos' columns representing positions.

    Returns:
    step_lengths: A pandas Series representing the length of each step.
    """
    # Calculate the step lengths between consecutive positions using Euclidean distance
    step_lengths = np.array([distance.euclidean(trajectory.iloc[i-1], trajectory.iloc[i]) for i in range(1, trajectory.shape[0])])   
    
    return pd.Series(step_lengths)

# WIDGETS

In [9]:
Radio_Button = pnw.RadioButtonGroup(name='Radio Button Group', width=340, options=['BM','CRW', 'LF'], value='BM')
n_steps = pnw.IntSlider(name='Number of steps', width=340, value=500, step=100, start=100, end=1000)
s_x_pos = pnw.IntInput(name='Starting pos_x', width=160, value=0, step=1, start=-100, end=100)
s_y_pos = pnw.IntInput(name='Starting pos_y', width=160, value=0, step=1, start=-100, end=100)
s_pos = pn.Row(s_x_pos,s_y_pos)
speed = pnw.IntSlider(name="speed", width=340, value=6, start=0, end=10)
metrics = pnw.Select(name="Metrics", width=340, value='PL', options=['PL', 'MSD', 'TAD'])
cauchy = pnw.FloatInput(name='Cauchy coefficient', width=160, value=0.5, step=0.1, start=0.1, end=0.9, format="0.0")
alpha = pnw.FloatInput(name='Alpha α', width=160, value=1, step=0.1, start=0.1, end=2, format="0.0")
location = pnw.IntSlider(name='Location', width=340, value=0, step=1, start=0, end=10)

In [24]:
@pn.depends(Radio_Button)
def display_radio_button(Radio_Button):
    """Show the widgets"""
    if Radio_Button == "CRW":
        return pn.Column(n_steps, s_pos, speed, cauchy, metrics)
    elif Radio_Button == "LF":
        return pn.Column(n_steps, s_pos, speed, pn.Row(cauchy, alpha), location, metrics)
    return pn.Column(n_steps, s_pos, speed, metrics)

def create_3d_plot(rw, times):
    """Create the graphic of the trajectory"""
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=rw[:, 0], y=rw[:, 1], z=times,
        marker=dict(size=2),      
        line = dict(width=2,  color='orange'),
        mode='lines',
    ))
    fig.update_layout(
        autosize=False,
        margin=dict(l=2, r=2, t=30, b=10),
    )
    return fig

def create_metric_plot(rw, times, metrics, cauchy, Radio_Button):
    """Generate the metric (PL, MSD o TAD)"""
    fig = go.Figure()
    
    if metrics == "TAD":
        traj_metric = trajectory_angles(rw)
        aux_domain = np.linspace(-np.pi, np.pi, 500)
        aux = np.mod(aux_domain, 2 * np.pi)
        wrapcauchy_pdf = np.array([wrapcauchy.pdf(i, cauchy) for i in aux])  

        fig.add_trace(go.Scatter(
            x=aux_domain, y=wrapcauchy_pdf,
            line=dict(width=2, color='red'),            
            mode='lines', name=f' {Radio_Button}, Cauchy {cauchy}'
        ))
        fig.add_trace(go.Histogram(
            x=traj_metric, nbinsx=150, histnorm='probability density',
            marker=dict(color='red'), opacity=0.5,
            name=f'Observed Cauchy {cauchy}'
        ))
    
    else:
        traj_metric = path_length(rw) if metrics == "PL" else msd(rw)
        fig.add_trace(go.Scatter(
            x=times, y=traj_metric,
            marker=dict(size=2), line=dict(width=2),
            mode='lines'
        ))
    fig.update_layout(
        autosize=False,
        width=500, height=300,
        margin=dict(l=2, r=2, t=30, b=10),
        legend=dict(
            orientation="h",  # Horizontal label
            yanchor="top", 
            y=-0.2  # Moves the labels to bottom
        ),
    )  
    
    return fig

@pn.depends(n_steps, speed, s_x_pos, s_y_pos, alpha, cauchy, location, Radio_Button, metrics)
def plot_traj(n_steps, speed, s_x_pos, s_y_pos, alpha, cauchy, location, Radio_Button, metrics):
    """ Generate the simulation and return the graphics on the panel"""
    times = np.linspace(0, 1, n_steps)

    if Radio_Button == "CRW":
        rw = CRW_2d(n_steps, speed, s_pos=[s_x_pos, s_y_pos], cauchy=cauchy)
    elif Radio_Button == "LF":
        rw = LevyW_2d(n_steps, speed, s_pos=[s_x_pos, s_y_pos], cauchy=cauchy, alpha=alpha, loc=location)
    else:
        rw = bm_2d(n_steps, speed, s_pos=[s_x_pos, s_y_pos])

    fig_RW_3d = create_3d_plot(rw, times)
    fig_metric = create_metric_plot(rw, times, metrics, cauchy, Radio_Button)

    return pn.Row(
        pn.Card(fig_RW_3d, sizing_mode='stretch_width', title=f'{Radio_Button} in 3D'),
        pn.Card(fig_metric, sizing_mode='stretch_width', title=f'{metrics}')
    )

# PANEL

In [26]:
# Template
a=pn.template.MaterialTemplate(
    site="HW 4",
    title="Dashboard",
    sidebar=[pn.Column(Radio_Button, display_radio_button)],
    main=[plot_traj]   
)


In [13]:
server = a.show(port=5006)

Launching server at http://localhost:5006


In [28]:
server.stop()
del server  # Elimina la referencia para evitar conflictos
server = a.show(port=5006)

Launching server at http://localhost:5006


ERROR:tornado.application:Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x0000022E25E5C1A0>>, <Task finished name='Task-3062' coro=<ServerSession.with_document_locked() done, defined at C:\Users\diana\OneDrive\Escritorio\VirtualEnviroments\hw1_mcap\Lib\site-packages\bokeh\server\session.py:77> exception=RuntimeError("Models must be owned by only a single document, ImportedStyleSheet(id='84a3cea1-4357-4618-ae87-b172faab8fbc', ...) is already in a doc")>)
Traceback (most recent call last):
  File "C:\Users\diana\OneDrive\Escritorio\VirtualEnviroments\hw1_mcap\Lib\site-packages\tornado\ioloop.py", line 750, in _run_callback
    ret = callback()
          ^^^^^^^^^^
  File "C:\Users\diana\OneDrive\Escritorio\VirtualEnviroments\hw1_mcap\Lib\site-packages\tornado\ioloop.py", line 774, in _discard_future_result
    future.result()
  File "C:\Users\diana\OneDrive\Escritorio\VirtualEnviroments\hw1_mcap\